## 19.12.25 word remover

In [17]:
from itertools import filterfalse


def word_remover(words):
    implies = dict.fromkeys(words, True).get # creating function str -> (True or None)
    remove_words= lambda doc: list(filterfalse(implies, doc))
    return remove_words

In [18]:
dict.fromkeys(['qwe','asd','zxc'], True).get

<function dict.get(key, default=None, /)>

In [5]:
# [T], a -> {T: a}
# arguments type: iterable and value
# return type: dict

l1 = ['qwe','asd','zxc']
v1 = 3.5
dict.fromkeys(l1, v1)

{'qwe': 3.5, 'asd': 3.5, 'zxc': 3.5}

In [6]:
l1 = ['qwe','asd','zxc']
v1 = 3.5
dict.fromkeys(l1)

{'qwe': None, 'asd': None, 'zxc': None}

In [7]:
dict.fromkeys(l1, True)

{'qwe': True, 'asd': True, 'zxc': True}

In [15]:
# how dict.get() works
dict.fromkeys(l1, True).get('qwe'), dict.fromkeys(l1, True).get('hoge', 'err') 

(True, 'err')

In [33]:
# jupyter notebook doesn't show None
dict.fromkeys(l1, True).get('hoge')

In [34]:
# to show None...
print(None)

None


In [35]:
# not None means True
type(None), not None

(NoneType, True)

In [23]:
True, False, not True, not False

(True, False, False, True)

In [22]:
type(True), type(False)

(bool, bool)

In [41]:
# creating function
# arg type: str
# return type: True or None
implies = dict.fromkeys(l1, True).get 
print(implies('wer'), implies('qwe'))

None True


In [46]:
# filter 
doc = ['qwe','zxc','wer','sdf','xcv']
list(filterfalse(implies, doc))

['wer', 'sdf', 'xcv']

In [47]:
# filter function
# arg type: [str]
# return type: [str]

remove_words= lambda doc: list(filterfalse(implies, doc))
doc = ['qwe','zxc','wer','sdf','xcv', 'ert','dfg','xcv','qwe','asd']
remove_words(doc)

['wer', 'sdf', 'xcv', 'ert', 'dfg', 'xcv']

# 19.12.25 random forest

## see white board

In [52]:
from functools import partial
from operator import attrgetter, getitem, invert, itemgetter, methodcaller

import numpy as np

def to_categorical(y, n_classes):
    """
    """
    # input_shape = y.shape
    n_samples = y.shape[0]
    categorical = np.full((n_samples, n_classes), 0, dtype=np.int8)
    categorical[np.arange(n_samples), y] = 1
    # output_shape = input_shape + (n_classes,)
    # return np.reshape(categorical, output_shape)
    return categorical

def best_splitter(max_features,
                  class_weight,
                  random_state=None,
                  criterion=lambda proba_stride: 1 - np.einsum('ij,ij->i', proba_stride, proba_stride)):

    np.random.seed(random_state)

    def node_split(X, y_encoded):

        n_features = X.shape[1]
        features = np.random.choice(n_features, max_features, replace=False)
        
        n_labels = np.einsum('ij->j', y_encoded) * class_weight
        n_samples = np.einsum('i->', n_labels)
        impurity = criterion((n_labels / n_samples)[np.newaxis, :])
        
        thresholds = [None] * max_features
        max_info_gains = [None] * max_features

        for i, x in enumerate(X.T[features]):

            edges, freq = np.unique(x, return_counts=True)
            
            if edges.shape[0] < 2:
                thresholds[i] = np.nan
                max_info_gains[i] = 0.
                
            else:
                rank = x.argsort()

                n_samples_stride_left = freq[:-1].cumsum()

                n_labels_stride_left = y_encoded[rank].cumsum(axis=0)[n_samples_stride_left - 1] * class_weight
                n_labels_stride_right = n_labels[np.newaxis, :] - n_labels_stride_left

                n_samples_stride_left = np.einsum('ij->i', n_labels_stride_left)
                n_samples_stride_right = np.einsum('ij->i', n_labels_stride_right)

                proba_stride_left = n_labels_stride_left / n_samples_stride_left[:, np.newaxis]
                proba_stride_right = n_labels_stride_right / n_samples_stride_right[:, np.newaxis]

                info_gains = criterion(proba_stride_right) * n_samples_stride_right
                info_gains += criterion(proba_stride_left) * n_samples_stride_left
                info_gains /= -n_samples
                info_gains += impurity

                idx = np.argmax(info_gains)
                thresholds[i] = np.mean(edges[idx:idx + 2])
                max_info_gains[i] = info_gains[idx]

        idx = np.argmax(max_info_gains)
        feature, threshold, info_gain = map(itemgetter(idx), (features, thresholds, max_info_gains))
        
        if np.isnan(threshold):
            feature = -1
            
        return feature, threshold, info_gain

    return node_split

def build_depth_first_tree(X,
                           y,
                           node_split,
                           max_depth=1,
                           min_samples_split=2,
                           min_samples_leaf=1,
                           min_impurity_decrease=0.):
    """
    """
    UNDEFINED = -1
    
    n_samples, n_features = X.shape
    n_labels = np.bincount(y)
    y_encoded = to_categorical(y, n_labels.shape[0]).astype(np.int64)
    
    stack = [(UNDEFINED, 0, 0, np.full(n_samples, True))]
    push = stack.append
    pop = stack.pop

    tree = []
    add_node = tree.append

    while stack != []:
        parent, depth, is_left, mask = pop()

        node_id = len(tree)
        n_samples = X[mask].shape[0]
        feature = UNDEFINED
        threshold = np.nan

        is_leaf = (depth >= max_depth or
                   n_samples < min_samples_split)

        if not is_leaf:
            feature, threshold, info_gain = node_split(X[mask], y_encoded[mask])
            
            if not np.isnan(threshold):
                condition = X[mask, feature] >= threshold
            
                is_leaf = (info_gain <= min_impurity_decrease or
                           min(condition.sum(), invert(condition).sum()) < min_samples_leaf)
            
            else:
                is_leaf = 1

            if is_leaf:
                feature, threshold = UNDEFINED, np.nan

        node = {
            'node_id': node_id,
            'parent': parent,
            'depth': depth,
            'is_leaf': int(is_leaf),
            'is_left': is_left,
            'feature': feature,
            'threshold': threshold,
            'n_labels': np.einsum('ij->j', y_encoded[mask]),
            'arity': 0
        }
        add_node(node)
        
        if parent != -1:
            tree[parent]['arity'] += 1

        if not is_leaf:
            condition = X[:, feature] >= threshold
            push((node_id, depth + 1, 0, mask & condition))
            push((node_id, depth + 1, 1, mask & invert(condition)))
            
    return np.asarray(tree)

get_tree_attribute = lambda tree, attr: np.asarray(list(map(itemgetter(attr), tree)))

def search_subtree(tree, start):
    """
    """
    stop = start + 1
    total = tree[start]['arity']
    
    while total > 0:
        total += tree[stop]['arity'] - 1
        stop += 1
        
    return slice(start, stop)

def prune_tree(tree, start):
    """
    """
    mask = np.full(tree.shape[0], False)
    mask[search_subtree(tree, start)] = True
    subtree = tree[mask]
    n_labels = np.einsum('ij->j', get_tree_attribute(subtree, 'n_labels'))
    
    node = {
        'node_id': start,
        'parent': subtree[0]['parent'],
        'depth': subtree[0]['depth'],
        'is_leaf': 1,
        'is_left': subtree[0]['is_left'],
        'feature': -1,
        'threshold': np.nan,
        'n_labels': n_labels,
        'arity': 0
    }
    
    return np.insert(tree[invert(mask)], start, node)

def compute_tree_impurity(tree,
                          class_weight,
                          criterion=lambda proba_stride: 1 - np.einsum('ij,ij->i', proba_stride, proba_stride)):
    """
    """
    leaf_nodes = tree[get_tree_attribute(tree, 'is_leaf') == 1]
    n_labels_stride = get_tree_attribute(leaf_nodes, 'n_labels') * class_weight
    n_samples_stride = np.einsum('ij->i', n_labels_stride)
    proba_stride = n_labels_stride / n_samples_stride[:, np.newaxis]
    impurity = np.einsum('i->', criterion(proba_stride) * n_samples_stride) / np.einsum('i->', n_samples_stride)

    return leaf_nodes.shape[0], impurity

def minimum_description_length(tree,
                               class_weight,
                               criterion=lambda proba_stride: 1 - np.einsum('ij,ij->i', proba_stride, proba_stride)):
    """
    """
    n_leaf_nodes_vs_tree_impurity  = []
    for i in range(tree.shape[0]):
        n_leaf_nodes_vs_tree_impurity.append(compute_tree_impurity(prune_tree(tree, i), class_weight, criterion))

    return np.asarray(n_leaf_nodes_vs_tree_impurity)